In [1]:
%matplotlib qt
import pyoptflight as pof
from pyoptflight import initialize as optinit
from pyoptflight import plotting as optplot
import numpy as np

In [2]:
kerbin = pof.Body("Kerbin")
kerbin.omega_0 = 0
vehicle = pof.Stage.load_vehicle('mintoc_multi')
# vehicle = pof.Stage.load_vehicle('mintoc_single')

config = pof.SolverConfig(landing=False, 
                          T_min      = 1,
                          T_max      = 700,
                          max_iter   = 10000, 
                          solver_tol = 1e-3, 
                          N          = 5, 
                          T_init     = 100,
                          integration_method = 'RK4')

x0 = pof.LatLngBound(lat=0, lng=0, alt=0, vel = 1e-6, ERA0=0)

xf = pof.KeplerianBound(i    = np.deg2rad(20),
                        Ω    = np.deg2rad(0),
                        ω    = 0,
                        ha   = 120,
                        hp   = 120,
                        body = kerbin)

msolver = pof.Solver(kerbin, vehicle, config, x0, xf)

In [ ]:
# msolver.update_constraints(constraint_names='max_tau', new_values=0.10, new_enables=True)
# msolver.update_constraints(constraint_names='max_body_rate_y', new_values=np.deg2rad(5), new_enables=True)
# msolver.update_constraints(constraint_names='max_body_rate_z', new_values=np.deg2rad(5), new_enables=True)
# msolver.update_constraints(constraint_names='f_min', new_values=0.15, new_enables=True)
# msolver.update_constraints(constraint_names='max_alpha', new_values=np.deg2rad(10), new_enables=True)
# msolver.update_constraints(constraint_names='max_q', new_values=0.2, new_enables=True)
msolver.constraints

In [3]:
msolver.create_nlp()

In [4]:
msolver.initialize_from_func(pof.gravity_turn, opts={'skew':True})

The minimum cost is 0.006308433884273157 rad
This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      482
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      276

Total number of variables............................:       97
                     variables with only lower bounds:       49
                variables with lower and upper bounds:       48
                     variables with only upper bounds:        0
Total number of equality constraints.................:       83
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  9.9711111e-01 1.44e+01 1.1

In [ ]:
msolver.stats()

In [ ]:
msolver.solve_nlp()

In [ ]:
msolver.stats()

In [ ]:
fig = pof.plot_qalpha(msolver)

In [ ]:
fig = pof.plot_control_rates(msolver)

In [ ]:
fig = pof.plot_throttle(msolver)
fig.show()

In [ ]:
fig = pof.plot_attitude(msolver)
fig.show()

In [ ]:
pof.plot_solutions(msolver, colorscale='vel', markers='vel', show_actual_orbit=True, show_target_orbit=True, indices=[-1])

In [ ]:
sol = msolver.stage_sols[-1][0]
x, u = np.array(sol.X), np.array(sol.U)[0:-1]
px, py, pz = x[:, 1], x[:, 2], x[:, 3]
vx, vy, vz = x[:, 4], x[:, 5], x[:, 6]
psi, theta = u[:, 1], u[:, 2]
ebx = np.block([[np.cos(psi)*np.cos(theta)], 
                        [np.sin(psi)*np.cos(theta)], 
                        [-np.sin(theta)]]).T
vel = np.block([[vx], [vy], [vz]]).T
wind = np.block([[-msolver.body.omega_0*py], [msolver.body.omega_0*px], [np.zeros_like(pz)]]).T
# alpha = np.arccos(np.sum(ebx*(vel[:-1] - wind[:-1]), axis=1)/np.sqrt(np.sum((vel[:-1] - wind[:-1])**2, axis=1))) #kN
alpha = np.arccos(np.sum(ebx*(vel[1:] - wind[1:]), axis=1)/np.sqrt(np.sum((vel[1:] - wind[1:])**2, axis=1))) #kN

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # registers the 3D projection
import ipywidgets as widgets
from IPython.display import display

# === Sample Data Setup ===
# Replace these with your actual arrays.
N = 100  # total number of samples per array
# Create dummy arrays for demonstration.
arr1 = np.random.rand(N, 3) * 2 - 1  # values between -1 and 1
arr2 = np.random.rand(N, 3) * 2 - 1

# Dictionary to hold your named arrays.
arrays = {
    'Vector A': arr1,
    'Vector B': arr2
}

# === Plot Setup ===
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

# # Add this event handler after creating the figure and ax, e.g. after initializing the plot:
def on_press(event):
    if event.inaxes == ax:
        on_press.initial_elev = ax.elev  # lock current elevation
        on_press.initial_azim = ax.azim
        on_press.x = event.x

fig.canvas.mpl_connect('button_press_event', on_press)

# On mouse motion, update only the azimuth while keeping the elevation constant.
def on_motion(event):
    if event.inaxes == ax and event.button is not None and hasattr(on_press, 'x'):
        dx = event.x - on_press.x
        # Update azimuth based on horizontal movement (tweak factor as needed)
        new_azim = on_press.initial_azim - dx * 0.5  
        ax.view_init(elev=on_press.initial_elev, azim=new_azim)
        fig.canvas.draw_idle()

fig.canvas.mpl_connect('motion_notify_event', on_motion)

def plot_vectors(index):
    ax.clear()
    # Plot the coordinate axes with arrows.
    ax.quiver(0, 0, 0, 1, 0, 0, color='r', arrow_length_ratio=0.1, label='X axis')
    ax.quiver(0, 0, 0, 0, 1, 0, color='g', arrow_length_ratio=0.1, label='Y axis')
    ax.quiver(0, 0, 0, 0, 0, 1, color='b', arrow_length_ratio=0.1, label='Z axis')
    
    # Plot each vector from the arrays at the given index.
    for name, arr in arrays.items():
        # Support negative indexing (e.g., -1 gives the last element)
        idx = index if index >= 0 else arr.shape[0] + index
        vector = arr[idx]
        ax.quiver(0, 0, 0, vector[0], vector[1], vector[2],
                  arrow_length_ratio=0.1, label=name)
    
    # Set axis limits so that the vectors and axes are clearly visible.
    ax.set_xlim([-1, 1])
    ax.set_ylim([-1, 1])
    ax.set_zlim([-1, 1])
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.legend()
    plt.draw()

# === Widgets Setup ===
# Slider to scrub through index values. We allow negative indices.
slider = widgets.IntSlider(value=0, min=-N, max=N-1, description='Index:')
# A number input for manual entry.
number_input = widgets.IntText(value=0, description='Index Input:')

# Synchronize the slider and the text input.
def update_slider(change):
    new_val = change['new']
    if number_input.value != new_val:
        number_input.value = new_val
    plot_vectors(new_val)
    update_output(new_val)

def update_text(change):
    new_val = change['new']
    if slider.value != new_val:
        slider.value = new_val
    plot_vectors(new_val)
    update_output(new_val)

slider.observe(update_slider, names='value')
number_input.observe(update_text, names='value')

# === Output Area for Vector Details ===
output = widgets.Output()

def update_output(index):
    with output:
        output.clear_output()
        print(f"Vector details at index {index}:")
        for name, arr in arrays.items():
            idx = index if index >= 0 else arr.shape[0] + index
            vector = arr[idx]
            # Sample calculation: compute the magnitude of the vector.
            magnitude = np.linalg.norm(vector)
            print(f"{name}: x = {vector[0]:.3f}, y = {vector[1]:.3f}, z = {vector[2]:.3f} | |v| = {magnitude:.3f}")

# === Initialize Display ===
plot_vectors(0)
update_output(0)

# Display the interactive widgets and output.
display(widgets.HBox([slider, number_input]))
display(output)
plt.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# === Sample Data Setup ===
N = 100  # total number of samples per array

# Dictionary to hold your named arrays.
arrays = {
    'Velocity': vel[:-1],
    'ebx': ebx,
    'R': x[:, 1:4][:-1]
}

# === Function to Create the 3D Plotly Figure ===
def create_figure(index):
    # Create a new Plotly figure.
    fig = go.Figure()
    
    # Add coordinate axes (as lines from the origin).
    fig.add_trace(go.Scatter3d(
        x=[0, 1], y=[0, 0], z=[0, 0],
        mode='lines',
        line=dict(color='red', width=4),
        name='X axis'
    ))
    fig.add_trace(go.Scatter3d(
        x=[0, 0], y=[0, 1], z=[0, 0],
        mode='lines',
        line=dict(color='green', width=4),
        name='Y axis'
    ))
    fig.add_trace(go.Scatter3d(
        x=[0, 0], y=[0, 0], z=[0, 1],
        mode='lines',
        line=dict(color='blue', width=4),
        name='Z axis'
    ))
    
    # For each vector array, add a trace to draw a line (with markers) from the origin
    # to the vector at the given index.
    for name, arr in arrays.items():
        idx = index if index >= 0 else arr.shape[0] + index  # support negative indexing
        vec = arr[idx]
        fig.add_trace(go.Scatter3d(
            x=[0, vec[0]],
            y=[0, vec[1]],
            z=[0, vec[2]],
            mode='lines+markers',
            marker=dict(size=4),
            line=dict(width=5),
            name=name
        ))
    
    # Update layout: fix the camera's up direction so the z‑axis stays up.
    fig.update_layout(
        scene=dict(
            xaxis=dict(range=[-1, 1]),
            yaxis=dict(range=[-1, 1]),
            zaxis=dict(range=[-1, 1]),
            camera=dict(
                up=dict(x=0, y=0, z=1)
            )
        ),
        margin=dict(l=0, r=0, b=0, t=0),
        showlegend=True
    )
    return fig

# === Initialize the Plotly Figure ===
current_index = 0
fig = create_figure(current_index)
fig_widget = go.FigureWidget(fig)

# === Widgets Setup ===
# Slider to scrub through index values. We allow negative indices.
slider = widgets.IntSlider(value=0, min=-N, max=N-1, description='Index:')
# A number input for manual entry.
number_input = widgets.IntText(value=0, description='Index Input:')

# Update function for the slider.
def update_plot(change):
    new_index = slider.value
    if number_input.value != new_index:
        number_input.value = new_index
    new_fig = create_figure(new_index)
    with fig_widget.batch_update():
        # Update each trace's coordinates in the existing figure widget.
        for i, trace in enumerate(new_fig.data):
            fig_widget.data[i].update(x=trace.x, y=trace.y, z=trace.z)
        fig_widget.layout.update(new_fig.layout)
    update_output(new_index)

def update_text(change):
    new_index = number_input.value
    if slider.value != new_index:
        slider.value = new_index
    new_fig = create_figure(new_index)
    with fig_widget.batch_update():
        for i, trace in enumerate(new_fig.data):
            fig_widget.data[i].update(x=trace.x, y=trace.y, z=trace.z)
        fig_widget.layout.update(new_fig.layout)
    update_output(new_index)

slider.observe(update_plot, names='value')
number_input.observe(update_text, names='value')

# === Output Area for Vector Details ===
output = widgets.Output()
def update_output(index):
    with output:
        output.clear_output()
        print(f"Vector details at index {index}:")
        for name, arr in arrays.items():
            idx = index if index >= 0 else arr.shape[0] + index
            vector = arr[idx]
            magnitude = np.linalg.norm(vector)
            print(f"{name}: x = {vector[0]:.3f}, y = {vector[1]:.3f}, z = {vector[2]:.3f} | |v| = {magnitude:.3f}")
        idx = index if index >= 0 else N + index
        vel = arrays['Velocity'][idx]
        ebx = arrays['ebx'][idx]
        aoa = np.rad2deg(np.arccos(min(np.dot(vel, ebx)/np.linalg.norm(vel), 1)))
        print(f"AoA: {aoa:.3f} deg")
        print(f"f: {u[idx][0]:.3f}")

update_output(current_index)

# === Display Everything ===
display(widgets.HBox([slider, number_input]))
display(output)
fig_widget